In [30]:
import pandas as pd
import numpy as np

In [31]:
# download best model
import wandb
api = wandb.Api()
run = api.run("deep_learning_hsa/Abgabe_02/alq2sz0n")
model=run.file("model-best.h5").download(replace=True)

In [32]:
from sharedtask02.Model.model import create_traindataset
from sklearn.preprocessing import StandardScaler
# Import Data
from sklearn.preprocessing import MinMaxScaler
# Read in the csv data using pandas
train = pd.read_csv('../../data/01_train/train.csv', header=None)

#sample_y
sampleTest = pd.read_csv('../../data/01_train/sampleSubmission.csv', header=None)
sampleTest=sampleTest.to_numpy()

#sample_x
test_original = pd.read_csv('../../data/01_train/sampleTest.csv', header=None)
test_original=test_original.to_numpy()
test=test_original.copy()

#Abgabe_x
Abgabe_x = pd.read_csv('../../data/02_test/test_x.csv', header=None)
Abgabe_x=Abgabe_x.to_numpy()

In [33]:
x_train, y_train, x_test, y_test = create_traindataset(data=train)
scaler = StandardScaler()
#scal Data

scaler.fit_transform(x_train.reshape(-1, 1))

shape = x_train.shape
x_train = scaler.transform(x_train.reshape(-1, 1))
x_train = x_train.reshape(shape)

shape = x_test.shape
x_test = scaler.transform(x_test.reshape(-1, 1))
x_test = x_test.reshape(shape)

shape = test.shape
test = scaler.transform(test.reshape(-1, 1))
test = test.reshape(shape)

shape = Abgabe_x.shape
Abgabe_x = scaler.transform(Abgabe_x.reshape(-1, 1))
Abgabe_x = Abgabe_x.reshape(shape)


print("Data successful preprocessed!")

Data successful preprocessed!
Data successful preprocessed!


In [34]:
#test if valide
test_reinitalisiert=scaler.inverse_transform(test)

if np.array_equal(test_original,test_reinitalisiert):
    print('Erfolg')
else:
    print('Rundungsfehler')
    for r in range(test_original.shape[0]):
        for c in range(test_original.shape[1]):
            if test_original[r, c] != round(test_reinitalisiert[r, c],10):
                print(r,c,test_original[r,c],test_reinitalisiert[r,c])


Rundungsfehler
Rundungsfehler


In [35]:
import keras

model = keras.models.load_model('model-best.h5')

In [36]:
import tensorflow as tf
test = np.expand_dims(np.array(test), axis=-1)
test = tf.convert_to_tensor(test)

Abgabe_x = np.expand_dims(np.array(Abgabe_x), axis=-1)
Abgabe_x = tf.convert_to_tensor(Abgabe_x)


In [37]:
from sklearn.metrics import mean_squared_error

print('pred_sample')
pred_sample = model.predict(x=test)
print(mean_squared_error(pred_sample, sampleTest))
print('')

print('train')
pred_train = model.predict(x=x_train)
y_train =    y_train.reshape(pred_train.shape)
#pred_train = np.expand_dims(np.array(pred_train), axis=-1)
print(mean_squared_error(pred_train, y_train))
print('')

print('test')
pred_test = model.predict(x=x_test)
y_test =    y_test.reshape(pred_test.shape)
print(mean_squared_error(pred_test, y_test))

print('test')
pred_Abgabe = model.predict(x=Abgabe_x)


pred_sample
1/1 [==============================] - 0s 175ms/step
11.426252011441601

train
1555/1555 [==============================] - 6s 4ms/step
11.255100435838918

test
445/445 [==============================] - 2s 4ms/step
10.254463556884433
test
4/4 [==============================] - 0s 3ms/step
pred_sample
1/1 [==============================] - 0s 175ms/step
11.426252011441601

train
1555/1555 [==============================] - 6s 4ms/step
11.255100435838918

test
445/445 [==============================] - 2s 4ms/step
10.254463556884433
test
4/4 [==============================] - 0s 5ms/step


In [38]:
x=tf.concat([x_train,x_test],0)
y_pred = model.predict(x)
#y_pred= tf.convert_to_tensor(np.expand_dims(y_pred, axis=-1))

y = tf.concat([y_train,y_test],0)

mse = mean_squared_error(y_pred, np.squeeze(y.numpy()))
print(mse)

2000/2000 [==============================] - 7s 4ms/step
11.032736693496252
2000/2000 [==============================] - 9s 4ms/step
11.032736693496252


In [39]:
#sample
pred_sample_ausgabe=np.ndarray.tolist(pred_sample)
#abgabe
pred_Abgabe_ausgabe=np.ndarray.tolist(pred_Abgabe)

In [40]:
# predict Data

#pred_sampleTest= test.append(pred_sampleTest)
np.savetxt("../../data/01_train/pred_sample.csv",
           pred_sample_ausgabe,
           delimiter =", ",
           fmt ='% s')
print("pred_test.csv successful saved!")

np.savetxt("../../data/02_test/pred_Abgabe.csv",
           pred_Abgabe_ausgabe,
           delimiter =", ",
           fmt ='% s')

print("pred_Abgabe.csv successful saved!")

pred_test.csv successful saved!
pred_Abgabe.csv successful saved!
pred_test.csv successful saved!
pred_Abgabe.csv successful saved!
